# 0.3 Napari

## Set up

In [1]:
%run ./0.2-jvs-before_to_start.ipynb

## Libraries

In [2]:
from cell_viewer.UJ.UJExperiment import UJExperiment
from pprint import pprint
from copy import deepcopy
import napari

## Load experiment data

### Experiment directory

In [3]:
experiment_dir = data_raw_dir(
    "uJ_data",
    "pBGT-AMP-Pulse",
    "20190325_Bruno_pBGT_quimiostato"
)
experiment_dir.exists() and experiment_dir.is_dir()

True

### Read into UJExperiment object

In [4]:
experiment = UJExperiment(
    name=experiment_dir.name,
    path=experiment_dir
)

experiment.name, experiment.path.exists()

('20190325_Bruno_pBGT_quimiostato', True)

## Identify experiment traps

In [5]:
experiment_traps = experiment.trap_data("xy*")
experiment_traps.keys()

dict_keys(['xy23', 'xy35', 'xy46', 'xy20', 'xy25', 'xy19', 'xy41', 'xy13', 'xy02', 'xy34', 'xy32', 'xy14', 'xy42', 'xy31', 'xy15', 'xy33', 'xy26', 'xy05', 'xy01', 'xy22', 'xy16', 'xy06', 'xy27', 'xy07', 'xy43', 'xy21', 'xy38', 'xy47', 'xy11', 'xy12', 'xy03', 'xy29', 'xy10', 'xy36', 'xy30', 'xy17', 'xy37', 'xy44', 'xy45', 'xy28', 'xy39', 'xy18', 'xy08', 'xy09', 'xy24', 'xy40', 'xy04'])

## Select a trap

In [6]:
xy_trap = experiment_traps.get("xy02")
xy_trap

TrapExperiment(data_cells=PosixPath('/Users/jvelezmagic/Documents/Github/raflab/cell_viewer/notebooks/../data/raw/uJ_data/pBGT-AMP-Pulse/20190325_Bruno_pBGT_quimiostato/data_cells/xy02'), data_cells_tracked=PosixPath('/Users/jvelezmagic/Documents/Github/raflab/cell_viewer/notebooks/../data/raw/uJ_data/pBGT-AMP-Pulse/20190325_Bruno_pBGT_quimiostato/data_cells_tracked/xy02'), data_masks=PosixPath('/Users/jvelezmagic/Documents/Github/raflab/cell_viewer/notebooks/../data/raw/uJ_data/pBGT-AMP-Pulse/20190325_Bruno_pBGT_quimiostato/data_masks/xy02'), data_raw=PosixPath('/Users/jvelezmagic/Documents/Github/raflab/cell_viewer/notebooks/../data/raw/uJ_data/pBGT-AMP-Pulse/20190325_Bruno_pBGT_quimiostato/data_raw/xy02'), data_rois=PosixPath('/Users/jvelezmagic/Documents/Github/raflab/cell_viewer/notebooks/../data/raw/uJ_data/pBGT-AMP-Pulse/20190325_Bruno_pBGT_quimiostato/data_rois/xy02'), data_segmentable=PosixPath('/Users/jvelezmagic/Documents/Github/raflab/cell_viewer/notebooks/../data/raw/uJ_da

## Read tifs from specific trap

In [7]:
xy_tifs = xy_trap.read_tifs(search_on = None, compute = False)
xy_tifs.keys()

dict_keys(['data_masks', 'data_raw', 'data_segmentable', 'data_tif'])

In [8]:
xy_tifs = xy_trap.read_tifs(search_on=["data_masks"])
xy_tifs

{'data_masks': {'GFP': dask.array<concatenate, shape=(25, 512, 640), dtype=uint8, chunksize=(1, 512, 640), chunktype=numpy.ndarray>}}

## Read cells

In [9]:
xy_cells = xy_trap.read_cells(search_on=["data_cells_tracked"])
xy_cells.keys()

dict_keys(['data_cells_tracked'])

## Create CellViewerWidgets

In [10]:
import datetime
import numpy
from enum import Enum
from pathlib import Path
from magicgui import magicgui

class Medium(Enum):
    """Using Enums is a great way to make a dropdown menu."""
    Glass = 1.520
    Oil = 1.515
    Water = 1.333
    Air = 1.0003

@magicgui(
    call_button="Calculate",
    layout="vertical",
    result_widget=True,
    # numbers default to spinbox widgets, but we can make
    # them sliders using the `widget_type` option
    slider_float={"widget_type": "FloatSlider", "max": 100},
    slider_int={"widget_type": "Slider", "readout": False},
    radio_option={
        "widget_type": "RadioButtons",
        "orientation": "horizontal",
        "choices": [("first option", 1), ("second option", 2)],
    },
    filename={"label": "Pick a file:"},  # custom label
)
def widget_demo(
    boolean=True,
    integer=1,
    spin_float=3.14159,
    slider_float=43.5,
    slider_int=550,
    string="Text goes here",
    dropdown=Medium.Glass,
    radio_option=2,
    date=datetime.date(1999, 12, 31),
    time=datetime.time(1, 30, 20),
    datetime=datetime.datetime.now(),
    filename=Path.home(),  # path objects are provided a file picker
):
    """Run some computation."""
    return locals().values()

## Run napari

In [11]:
%gui qt5
viewer = napari.Viewer()

for directory, tifs in xy_tifs.items():
    for name, tif in tifs.items():

        if "gfp" in name.lower():
            colormap = "green"
            opacity = 1.0
        elif "red" in name.lower():
            colormap = "red"
            opacity = 1.0
        else:
            colormap = "gray"
            opacity = 0.1

        viewer.add_image(
            data=tif,
            name=f"{directory}_{name}",
            contrast_limits=[0, 2000],
            multiscale=False,
            blending="additive",
            colormap=colormap,
            opacity=opacity
        )
        
viewer.window.add_dock_widget(widget_demo, area="right")